In [ ]:
import math
import logging

from pprint import pprint
from multiprocessing import Pool

from default import *
from utils.common import compute_rtts_per_dst_src, compute_geo_info, compute_error, round_based_algorithm
from utils.file_utils import load_json, dump_json
from utils.helpers import select_best_guess_centroid, haversine, is_within_cirle

## Step 1

## Step X

In [ ]:
anchors = load_json(ANCHORS_FILE)
all_probes = load_json(PROBES_AND_ANCHORS_FILE)


vp_coordinates_per_ip, ip_per_coordinates, country_per_vp, asn_per_vp, \
    vp_distance_matrix, probe_per_ip = compute_geo_info(
        all_probes, serialized_file=PAIRWISE_DISTANCE_FILE)

In [ ]:

filter = ""
greedy_probes = load_json(GREEDY_PROBES_FILE)
rtt_per_srcs_dst = compute_rtts_per_dst_src(
    PROBES_TO_ANCHORS_PING_TABLE, filter, threshold=100)
tier1_vps_l = [10, 100, 300, 500, 1000]
error_cdf_per_tier1_vps = {}

for tier1_vps in tier1_vps_l:
    pprint(f"Using {tier1_vps} tier1_vps")
    error_cdf = round_based_algorithm(greedy_probes, rtt_per_srcs_dst, vp_coordinates_per_ip,
                                        asn_per_vp,
                                        tier1_vps,
                                        threshold=40)
    error_cdf_per_tier1_vps[tier1_vps] = error_cdf
dump_json(error_cdf_per_tier1_vps, ROUND_BASED_ALGORITHM_FILE)